In [ ]:
from katabatic.models.TableGAN import TableGANAdapter, TableGAN, preprocess_data, postprocess_data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Initialize the adapter with a specific privacy setting
tablegan_adapter = TableGANAdapter(type='continuous', privacy_setting='high')
data_path = 'data/shuttle/shuttle.tst'
df = pd.read_csv(data_path)
print(df[3:10])

2025-04-04 16:43:16,165 - katabatic.models.TableGAN - INFO - TableGAN module initialized


   55 0 81 0 -6 11 25 88 64 4
3   55 2 82 0 54 -6 26 28 2 1
4   41 0 84 3 38 -4 43 45 2 1
5  37 0 100 0 36 -8 63 64 2 1
6    46 0 83 0 46 0 37 36 0 1
7  44 0 79 0 42 -17 35 37 2 1
8   44 -1 78 0 44 0 34 34 0 1
9  55 0 81 0 54 -10 25 26 2 1


In [45]:
x = df.copy().drop('Class', axis=1)
y = df['Class']

x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

In [46]:
tablegan_adapter.fit(x_train, y_train, epochs=100, batch_size=64)

---FIT TableGAN Model with high privacy setting
---Initialise TableGAN Model


d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 10/100: [D loss: -0.9438] [G loss: 0.6501] [C loss: 0.6939]
Epoch 20/100: [D loss: -1.2742] [G loss: 1.1586] [C loss: 0.4941]
Epoch 30/100: [D loss: -1.1440] [G loss: 1.1257] [C loss: 0.3956]
Epoch 40/100: [D loss: -1.2232] [G loss: 1.3256] [C loss: 0.3590]
Epoch 50/100: [D loss: -1.1127] [G loss: 1.3535] [C loss: 0.3251]
Epoch 60/100: [D loss: -1.0771] [G loss: 1.3517] [C loss: 0.3015]
Epoch 70/100: [D loss: -1.1604] [G loss: 1.3237] [C loss: 0.2645]
Epoch 80/100: [D loss: -1.1383] [G loss: 1.3875] [C loss: 0.2270]
Epoch 90/100: [D loss: -1.0705] [G loss: 1.3176] [C loss: 0.1878]
Epoch 100/100: [D loss: -1.1155] [G loss: 1.3207] [C loss: 0.1527]


In [47]:
# Generate synthetic data
synthetic_data = tablegan_adapter.generate(size=1000)

---Generate from TableGAN Model


In [ ]:
synthetic_df = pd.DataFrame(synthetic_data)
x_sync_train = synthetic_df.drop(synthetic_df.columns[-1],axis=1).values
y_sync_train = synthetic_df.iloc[ :, -1:].values

In [58]:
# TSTR (train synthetic test real)
tstr_score_lr  = LogisticRegression().fit(x_sync_train, y_sync_train).score(x_test, y_test)
tstr_score_rf  = RandomForestClassifier().fit(x_sync_train, y_sync_train).score(x_test, y_test)
tstr_score_mlp = MLPClassifier().fit(x_sync_train, y_sync_train).score(x_test, y_test)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_sync_train = le.fit_transform(y_sync_train)

xgbt_classifier = XGBClassifier(eval_metric='logloss', use_label_encoder=True )
tstr_score_xgbt = xgbt_classifier.fit(x_sync_train, y_sync_train).score(x_test, y_test)


# TRTR (train real test real)
trtr_score_lr  = LogisticRegression().fit(x_train, y_train).score(x_test, y_test)
trtr_score_rf  = RandomForestClassifier().fit(x_train, y_train).score(x_test, y_test)
trtr_score_mlp = MLPClassifier().fit(x_train, y_train).score(x_test, y_test)
xgbt_classifier = XGBClassifier(eval_metric='logloss', use_label_encoder=True)
trtr_score_xgbt = xgbt_classifier.fit(x_train, y_train).score(x_test, y_test)
df_evaluate = pd.DataFrame([
    ['TSTR', tstr_score_lr, tstr_score_rf, tstr_score_mlp, tstr_score_xgbt],
    ['TRTR', trtr_score_lr,trtr_score_rf,trtr_score_mlp, trtr_score_xgbt]
], columns=['Evaluated Item', 'LR', 'RF', 'MLP', 'XGBT'])
print(df_evaluate)

d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataCo

  Evaluated Item        LR        RF       MLP      XGBT
0           TSTR  0.679191  0.684971  0.679191  0.089595
1           TRTR  0.658960  0.965318  0.936416  0.979769
